# **Predicción de Precios de Acciones de NVIDIA (NVDA) con Deep Learning**

### **Proyecto Final Samsung Innovation Campus – Módulo de Inteligencia Artificial (EC04)**

Integrantes:

- Ulices Chingo
- Sofia Feijóo
- Brayan Maisincho
- Esteban Quiña
- Alan Palma


In [1]:
# Librerías básicas para cómputo numérico y manejo de datos
import numpy as np
import pandas as pd
import warnings
import itertools

# Librerías para visualización
import matplotlib.pyplot as plt

# Librerías para análisis estadístico y series temporales
import statsmodels.api as sm

# Librerías de scikit-learn para preprocesamiento y evaluación
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Módulos propios del proyecto
from src.model_LSTM import model_LSTM
from src.indicators import agregar_indicadores_tecnicos

# LSTM

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [3]:
# Configuracion para el modelo
ventana = 60
epocas = 20
batch_size = 32

In [6]:

# Crarga de datos y preprocesamiento
df = pd.read_csv("valid_csv_concatenado_invertido_con_60min.csv", parse_dates=['timestamp'])

# Filtrar fechas de interés
df = df[(df['timestamp'] >= '2023-01-01') & (df['timestamp'] <= '2023-06-30')]

# Agrgar indicadores técnicos
df = agregar_indicadores_tecnicos(df)

# Eliminar filas con NaN generados por los indicadores (los primeros 26 periodos del MACD/RSI)
df.dropna(inplace=True)

# Seleccionar FEATURES (X) y TARGET (y): se usara Close, RSI y MACD para predecir.
features = ['close', 'rsi', 'macd']
target = 'close'

data = df[features].values

# Scalamiento multivariado: 3 columnas (close, rsi, macd)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Para poder invertir la predicción del precio después, necesitamos un scaler solo para el precio (columna target)
scaler_precio = MinMaxScaler()
scaler_precio.fit(df[[target]]) # Ajustamos solo a la columna target para uso posterior


# ====== Split ======
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size - ventana:]

# Crear secuencias multivariadas
def crear_secuencias_multivariadas(data, ventana, target_col_idx=0):
    X, y = [], []
    for i in range(ventana, len(data)):
        # X: Todas las columnas (precio, rsi, macd) de los 'ventana' días anteriores
        X.append(data[i - ventana:i, :]) 
        
        # y: Solo la columna del precio (target_col_idx) del día actual 'i'
        y.append(data[i, target_col_idx]) 
        
    return np.array(X), np.array(y)

# Usar la nueva función
# target_col_idx=0 porque 'close' es la primera columna en nuestra lista ['close', 'rsi', 'macd']
X, y = crear_secuencias_multivariadas(scaled_data, ventana, target_col_idx=0)

# Split Train/Test (Mantenemos tu lógica 80/20)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"Forma de X_train: {X_train.shape}") 
# Debería ser (N muestras, 60 ventana, 3 features) -> Antes era (N, 60, 1)



Forma de X_train: (1526, 60, 3)


In [ ]:
# Usar model_LSTM
window = 60
batch_size = 32
epochs = 50

# Llamar a la función model_LSTM con los parámetros definidos
model = model_LSTM(window_size=window, batch_size=batch_size, epochs=epochs, X_train=X_train, y_train=y_train)

Epoch 1/50


2025-12-11 16:01:15.128802: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/alanp/anaconda3/envs/py310/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0090
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 3.3461e-04
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 1.0923e-04
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 1.0616e-04
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.0430e-04
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.0361e-04
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 1.0300e-04
Epoch 8/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.0404e-04
Epoch 9/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.0063e-04
Epoch 10/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.0032e-04
Epoch 11/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 9.9799e-05
Epoch 12/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 9.6722e-05
Epoch 13/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 9.7224e-05
Epoch 14/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 9.4630e-05
Epoch 15/50
48/48 ━━━━━━━━━━━━

In [ ]:
# Evaluar el modelo LSTM
predictions_LSTM = model.predict(X_test)
predictions_inverse_LSTM = scaler.inverse_transform(predictions_LSTM.reshape(-1, 1))
y_test_inverse_LSTM = scaler.inverse_transform(y_test.reshape(-1, 1))

# Metricas
mae = mean_absolute_error(y_test_inverse_LSTM, predictions_inverse_LSTM)
mse = mean_squared_error(y_test_inverse_LSTM, predictions_inverse_LSTM)
mape = np.mean(np.abs((y_test_inverse_LSTM - predictions_inverse_LSTM) / y_test_inverse_LSTM)) * 100

print(f"LSTM mean absolute error: {mae}")
print(f"LSTM mean squared error: {mse}")
print(f"LSTM mean absolute percentage error: {mape}")

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
LSTM mean absolute error: 0.4397710185481813
LSTM mean squared error: 0.5504371110160029
LSTM mean absolute percentage error: 1.0981321360087177
